In [1]:
import pandas as pd

In [97]:
#train = pd.read_csv('train.csv',sep='\t')
#test = pd.read_csv('test.csv',sep='\t')
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [98]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def review_to_wordlist( review, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 6. WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(x) for x in words]
    words =  lemmatized_words  
    # 5. Return a list of words
    return(words)

In [99]:
import nltk.data
#nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=True ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


In [100]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set" )
for review in train['reviews']:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


In [101]:
print(len(sentences) ) #795538
print(sentences[0])
'''
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
'''
print(sentences[11])

17829
['born', 'thunderstorm', 'grew', 'overnight', 'played', 'alone', 'played', 'survived', 'hey', 'wanted', 'everything', 'never', 'like', 'love', 'come', 'light', 'wore', 'envy', 'hated', 'survived', 'one', 'way', 'ticket', 'place', 'demon', 'go', 'wind', 'change', 'nothing', 'ground', 'ever', 'grow', 'hope', 'lie', 'taught', 'cry', 'pillow', 'survived', 'still', 'breathing', 'still', 'breathing', 'still', 'breathing', 'still', 'breathing', 'alive', 'alive', 'alive', 'alive', 'found', 'solace', 'strangest', 'place', 'way', 'back', 'mind', 'saw', 'life', 'stranger', 'face', 'mine', 'one', 'way', 'ticket', 'place', 'demon', 'go', 'wind', 'change', 'nothing', 'ground', 'ever', 'grow', 'hope', 'lie', 'taught', 'cry', 'pillow', 'survived', 'still', 'breathing', 'still', 'breathing', 'still', 'breathing', 'still', 'breathing', 'alive', 'alive', 'alive', 'alive', 'took', 'still', 'breathing', 'took', 'still', 'breathing', 'took', 'still', 'breathing', 'took', 'still', 'breathing', 'took', 

In [102]:
################################################################
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [103]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model..." )
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2018-12-02 18:23:51,083 : INFO : collecting all words and their counts
2018-12-02 18:23:51,083 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-02 18:23:51,155 : INFO : PROGRESS: at sentence #10000, processed 353912 words, keeping 14995 word types
2018-12-02 18:23:51,206 : INFO : collected 19973 word types from a corpus of 654147 raw words and 17829 sentences
2018-12-02 18:23:51,210 : INFO : Loading a fresh vocabulary
2018-12-02 18:23:51,218 : INFO : effective_min_count=40 retains 1783 unique words (8% of original 19973, drops 18190)
2018-12-02 18:23:51,222 : INFO : effective_min_count=40 leaves 561833 word corpus (85% of original 654147, drops 92314)
2018-12-02 18:23:51,228 : INFO : deleting the raw counts dictionary of 19973 items
2018-12-02 18:23:51,228 : INFO : sample=0.001 downsamples 65 most-common words
2018-12-02 18:23:51,228 : INFO : downsampling leaves estimated 463671 word corpus (82.5% of prior 561833)
2018-12-02 18:23:51,236 : INFO : estim

Training model...


2018-12-02 18:23:51,284 : INFO : training model with 4 workers on 1783 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2018-12-02 18:23:51,662 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-12-02 18:23:51,662 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-12-02 18:23:51,682 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-12-02 18:23:51,682 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-12-02 18:23:51,682 : INFO : EPOCH - 1 : training on 654147 raw words (463741 effective words) took 0.4s, 1193664 effective words/s
2018-12-02 18:23:52,101 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-12-02 18:23:52,115 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-12-02 18:23:52,119 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-12-02 18:23:52,125 : INFO : worker thread finished; awaiting finish of 0

In [104]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-12-02 18:23:58,932 : INFO : precomputing L2-norms of word weight vectors
2018-12-02 18:23:58,947 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2018-12-02 18:23:58,947 : INFO : not storing attribute vectors_norm
2018-12-02 18:23:58,947 : INFO : not storing attribute cum_table
2018-12-02 18:23:59,010 : INFO : saved 300features_40minwords_10context


In [105]:
from gensim.models import Word2Vec 
model = Word2Vec.load("300features_40minwords_10context")
#
type(model.wv.syn0)
#numpy.ndarray
model.wv.syn0.shape
#(14961, 300)  total 15000 words
################################################################
model["flower"]
model["flower"].shape
#(300,) # Word vector dimensionality 
################################################################

2018-12-02 18:24:01,557 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-12-02 18:24:01,588 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-12-02 18:24:01,588 : INFO : setting ignored attribute vectors_norm to None
2018-12-02 18:24:01,588 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-12-02 18:24:01,588 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-12-02 18:24:01,588 : INFO : setting ignored attribute cum_table to None
2018-12-02 18:24:01,588 : INFO : loaded 300features_40minwords_10context
C:\Users\hejia\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.
C:\Users\hejia\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationW

(300,)

In [106]:
#From Words To Paragraphs, Attempt 1: Vector Averaging
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32") # (300,) 0
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)  #len(model.wv.index2word):14961
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [107]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
        counter =int(counter)
        #
        # Print a status message every 1000th review
       
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)) )
        # 
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
        #
        # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [108]:
set(model.wv.index2word) #len(set(model.wv.index2word)):14961

{'sheet',
 'deny',
 'nasty',
 'damned',
 'tongue',
 'worse',
 'else',
 'gonna',
 'hotel',
 'stopping',
 'law',
 'livin',
 'track',
 'gucci',
 'michael',
 'hang',
 'bury',
 'high',
 'dust',
 'girlfriend',
 'alabama',
 'killing',
 'want',
 'motherfuckin',
 'hush',
 'boat',
 'silence',
 'bend',
 'hollywood',
 'hour',
 'le',
 'freedom',
 'bam',
 'must',
 'ghost',
 'happening',
 'kissing',
 'alone',
 'ceiling',
 'tick',
 'ohh',
 'flow',
 'order',
 'yup',
 'blah',
 'miss',
 'ray',
 'free',
 'stone',
 'puerto',
 'mud',
 'judge',
 'lemon',
 'drag',
 'cover',
 'packed',
 'halo',
 'green',
 'smokin',
 'skrt',
 'leaf',
 'cause',
 'rapper',
 'double',
 'sea',
 'quit',
 'whip',
 'table',
 'as',
 'mine',
 'low',
 'seat',
 'carry',
 'millionaire',
 'burned',
 'run',
 'tonight',
 'wrist',
 'dare',
 'almost',
 'heat',
 'big',
 'lyric',
 'standin',
 'summer',
 'wit',
 'neighbor',
 'weather',
 'south',
 'summertime',
 'dime',
 'toy',
 'momma',
 'motherfucking',
 'self',
 'rope',
 'wild',
 'extra',
 'care

In [109]:
num_features #300
train["reviews"]
train["reviews"][1]   

"LISTEN TO THIS TRACK BITCH!\r\r\n(Roscoe Dash)\r\r\nGirl the way you're movin'\r\r\nGot me in a trance\r\r\nDJ turn me up\r\r\nLadies dis yo jam\r\r\nI'ma sip Moscato\r\r\nAnd you 'gon lose dem pants\r\r\nThen I'ma throw this money\r\r\nWhile you do it with no hands\r\r\nGirl drop it to the flo'\r\r\nI love the way yo booty go\r\r\nAll I want to do is sit back\r\r\nAnd watch you move\r\r\nAnd I'll proceed to throw this cash\r\r\n\r\r\n(Waka Flocka Flame)\r\r\n\r\r\nAll that ass\r\r\nIn yo jeans\r\r\nCan Wale beet?\r\r\nCan Roscoe skeet?\r\r\nLong hair she don't care\r\r\nWhen she walk she get stares\r\r\nBrown skin or a yellow-bone\r\r\nDJ this my favorite song\r\r\nSo I'ma make it thunderstorm\r\r\nFlood warning, Flocka yeah\r\r\nBlowin' fuck it I don't care\r\r\nDreads is flyin' everywhere\r\r\nTap my partner Roscoe, like bruh\r\r\nI'm drunk as hell, can't you tell?\r\r\nThrew 70 bands, bet 50 stacks\r\r\nOh fuckin' well\r\r\nI'm tryna hit the hotel\r\r\nWith 2 girls that swallow me

In [110]:
clean_train_reviews = []
for review in train["reviews"]:
    clean_train_reviews.append( review_to_wordlist(review,remove_stopwords=True))
   

len(clean_train_reviews)   #25000 
######


3759

In [111]:
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )


Review 0 of 3759


C:\Users\hejia\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 3759
Review 2000 of 3759
Review 3000 of 3759


In [112]:
print("Creating average feature vecs for test reviews")    
#######
clean_test_reviews = []
for review in test["reviews"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )  #(25000, 300)   

Creating average feature vecs for test reviews
Review 0 of 940


C:\Users\hejia\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [113]:
from numpy import *

where_are_NaNs = isnan(testDataVecs)
testDataVecs[where_are_NaNs] = 0

In [114]:
testDataVecs[107]

array([-0.01340841,  0.00595989,  0.01482137, -0.01143448,  0.02157097,
        0.00666081,  0.02375888, -0.02832483,  0.03222426, -0.02747226,
        0.00675844,  0.03860399,  0.01065748, -0.01007283,  0.03086958,
        0.02839373,  0.00538496, -0.00528945,  0.01616366, -0.01699611,
        0.00182417,  0.0172285 , -0.02264713,  0.00270532,  0.03890445,
       -0.03839942, -0.01861985, -0.02068013,  0.00637237, -0.04703647,
       -0.05112464,  0.02291659, -0.00913461, -0.0008771 ,  0.01761519,
       -0.01009033,  0.02369462,  0.00496734,  0.02200226, -0.0268716 ,
        0.02912227,  0.02382296,  0.04993768,  0.02264572, -0.03545407,
       -0.00762905,  0.01159638, -0.00324208, -0.02374886, -0.04339325,
       -0.00588375, -0.00946959, -0.01977846,  0.02112148,  0.02084436,
        0.00900697,  0.01156769, -0.01511526, -0.03671454, -0.01650329,
        0.01804994,  0.05074625, -0.00567346, -0.01010526,  0.00885756,
       -0.00551338, -0.01761179,  0.00072385,  0.01333967,  0.02

In [115]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


def score(classifier):
    classifier = classifier
    aaa = classifier.fit(trainDataVecs, train["sentiment"])
    result = aaa.predict( testDataVecs )
    accuracy_score(test["sentiment"],result)
    return accuracy_score(test["sentiment"],result)


In [116]:
import sklearn
from nltk.classify.scikitlearn import  SklearnClassifier
from sklearn.svm import SVC, LinearSVC,  NuSVC
from sklearn.naive_bayes import  MultinomialNB, BernoulliNB
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import  accuracy_score

In [117]:
print('BernoulliNB`s accuracy is %f'  %score(BernoulliNB()))
#print('MultinomiaNB`s accuracy is %f'  %score(MultinomialNB()))
print('LogisticRegression`s accuracy is  %f' %score(LogisticRegression()))
print('SVC`s accuracy is %f'  %score(SVC()))
print('LinearSVC`s accuracy is %f'  %score(LinearSVC()))
print('NuSVC`s accuracy is %f'  %score(NuSVC()))

BernoulliNB`s accuracy is 0.562766
LogisticRegression`s accuracy is  0.551064


C:\Users\hejia\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hejia\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC`s accuracy is 0.500000
LinearSVC`s accuracy is 0.558511


C:\Users\hejia\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


NuSVC`s accuracy is 0.524468


In [21]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
aaa = classifier.fit(a, train["sentiment"])
result = aaa.predict( b )
accuracy_score(test["sentiment"],result)
#0.85011999999999999

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [40]:
import math

In [23]:
b.isnull().any()

AttributeError: 'list' object has no attribute 'isnull'

In [68]:
np.argwhere(np.isnan(trainDataVecs))

array([], shape=(0, 2), dtype=int64)

In [67]:
testDataVecs[107]

array([ -1.10830832e-02,   5.14353719e-03,   2.52754185e-02,
        -8.15838855e-03,   4.03351597e-02,   4.38734004e-03,
        -3.27906683e-02,  -2.87928395e-02,   2.90968250e-02,
         1.07309420e-03,   1.59700867e-02,  -7.32207031e-04,
        -6.17700303e-03,   5.00782346e-03,   2.61919544e-04,
         3.02386545e-02,  -3.43987631e-04,  -3.34602501e-03,
         1.83834862e-02,  -1.51501903e-02,  -1.90489516e-02,
         4.15357715e-03,   3.98910465e-03,   3.35276057e-03,
         1.14684459e-02,  -5.69698028e-03,  -2.88710967e-02,
         1.38502661e-02,   3.07408534e-02,  -8.53853300e-03,
        -2.56066248e-02,   1.41214579e-02,  -7.74361053e-03,
        -2.09821127e-02,   6.86992612e-03,  -8.40208866e-03,
         3.06519810e-02,   2.38238424e-02,  -9.61945206e-03,
        -1.31667415e-02,   4.94169667e-02,   6.85526896e-03,
         4.18798439e-02,   3.78956012e-02,  -3.18664387e-02,
        -1.90772526e-02,   3.51163186e-02,   1.15650697e-02,
        -3.26754525e-02,

In [61]:
np.argwhere(np.isfinite(trainDataVecs))

array([[   0,    0],
       [   0,    1],
       [   0,    2],
       ..., 
       [3758,  297],
       [3758,  298],
       [3758,  299]], dtype=int64)

In [62]:
trainDataVecs[np.isfinite(trainDataVecs)]  = 0

In [70]:
trainDataVecs[107]

array([  5.92918776e-04,   3.39077972e-02,   3.71629954e-03,
         1.76627059e-02,   1.87370889e-02,  -1.23408856e-02,
        -2.04783231e-02,  -1.81126859e-04,   6.25687232e-03,
        -5.04189054e-04,   1.43532492e-02,   6.15841942e-03,
        -3.65522504e-02,   4.67812503e-03,  -8.30571819e-03,
         2.03098059e-02,  -3.51084443e-03,  -1.42102120e-02,
        -7.64640979e-03,  -8.45149159e-03,   1.54138748e-02,
        -4.86446405e-03,   1.07870176e-02,   5.92428166e-03,
         1.70626212e-02,  -1.57537721e-02,  -1.17907636e-02,
        -1.46874990e-02,  -1.78152812e-03,  -2.04231459e-04,
        -1.89763438e-02,   1.97049743e-03,  -2.26746034e-02,
        -1.31292809e-02,   5.54790022e-03,  -1.18162399e-02,
         8.27252958e-03,   5.08068129e-03,  -4.41355212e-03,
         4.50796587e-03,   2.71568969e-02,  -8.01084097e-03,
         6.44361062e-05,   8.70372169e-03,  -2.72962265e-02,
         2.97690649e-03,  -2.06444203e-03,   1.78710353e-02,
        -8.21140409e-03,

In [118]:
testDataVecs.shape

(940, 300)